In [191]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [192]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}    # Create empty dict to hold alternate titles, if any
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',     # Loop through Keys that may hold alternate titles 
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:                                        # If any alternate title Keys have values ... 
            alt_titles[key] = movie[key]                        #   create a Key : Value entry in the alt_title dict
            movie.pop(key)                                      #   delete the original Key : Value entry in the movie dict
    if len(alt_titles) > 0:                                     # If any alt_titles are found and stored ...
        movie['alt_titles'] = alt_titles                        #   store the "alt_titles' dict in the original dict

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    
    return movie

In [193]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():

    kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv(f'{file_dir}ratings.csv')

    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
        wiki_movies_df = json.load(file)  # Load data from JSON file into a list of dicts

        
    return wiki_movies_df, kaggle_metadata, ratings
    
    # 3. Write a list comprehension to filter out TV shows.    
    with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)  # Load data from JSON file into a list of dicts

    #wiki_movies = [movie for movie in wiki_movies_raw if 'Television series' not in movie]
len(wiki_movies)


7310

In [359]:
# 3. Write a list comprehension to filter out TV shows.    
file_dir = "C:/Users/Bob Thomson/ETL Module/"
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)  # Load data from JSON file into a list of dicts

wiki_movies_raw = [movie for movie in wiki_movies_raw  # Perform list comprehension to eliminate TV shows
                   if 'Television series' not in movie]
len(wiki_movies_raw)

7310

In [360]:
# 4. Write a list comprehension to iterate through the cleaned wiki movies list
# and call the clean_movie function on each movie.
movies_Step4 = [clean_movie(movie) for movie in wiki_movies_raw] # Perform list comprehension to use cleaning function
# movies_Step4[1:3]  # Review the first two dicts in the list

In [361]:
# 5. Read in the cleaned movies list from Step 4 as a DataFrame.
movies_Step5_df=pd.DataFrame(movies_Step4)
#columns_Step4=sorted(movies_Step5_df.columns.tolist())  # Create list of column headers
#columns_Step4

In [362]:
movies_Step5_df.head(1) # Check latest DataFrame

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Number of employees,Divisions,Subsidiaries,Predecessor,Founders,Area served,Products,Services,Revenue,Operating income
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [363]:
# 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
#  dropping any imdb_id duplicates. If there is an error, capture and print the exception.

form_imdb="(tt\d{7})"

# For personal use - extract IMDB identifier from 'imdb_link' and store in 'imdb_id' column 
movies_Step5_df['imdb_id'] = movies_Step5_df['imdb_link'].str.extract(form_imdb)
movies_Step6_df=movies_Step5_df[movies_Step5_df['imdb_id'].notna()]

# for movies_Step5_df['imdb_id'] in movies_Step5_df['imdb_link']:
#     try:
#         movies_Step5_df['imdb_id'] = movies_Step5_df['imdb_link'].str.extract(form_imdb)
#         movies_Step5_df.drop_duplicates(subset='imdb_id', inplace=True)
#     except Exception as e:
#         print(e)
#         pass

len(movies_Step6_df)

7099

In [364]:
#  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
wiki_columns_to_keep = [column for column in movies_Step6_df.columns if movies_Step6_df[column].isnull().sum() < len(movies_Step6_df) * 0.9]
movies_Step7_df = movies_Step6_df[wiki_columns_to_keep]
movies_Step7_df.columns

Index(['url', 'year', 'imdb_link', 'title', 'Based on', 'Starring',
       'Cinematography', 'Release date', 'Running time', 'Country', 'Language',
       'Budget', 'Box office', 'Director', 'Distributor', 'Editor(s)',
       'Composer(s)', 'Producer(s)', 'Production company(s)', 'Writer(s)',
       'imdb_id'],
      dtype='object')

In [365]:
# 8. Create a variable (i.e. list???) that will hold the non-null values from the “Box office” column.
box_office=movies_Step7_df['Box office'].dropna() # Creates list of non-null contents from DataFrame
len(box_office)

5517

In [366]:
# 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [367]:
# 10. Write a regular expression to match the six elements of "form_one" of the box office data.
form_one = r'\$\d+\.?\d*\s*[mb]illion'

In [368]:
# 11. Write a regular expression to match the three elements of "form_two" of the box office data.
form_two = r'\$\d{1,3}(?:,\d{3})+'

In [369]:
# 12. Add the parse_dollars function.
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [370]:
# 13. Clean the box office column in the wiki_movies_df DataFrame.
# Use 'box_office' list to create new column in DataFrame
movies_Step7_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
# movies_Step7_df.columns
movies_Step7_df.drop('Box office', axis=1, inplace=True)
movies_Step13_df=movies_Step7_df
movies_Step13_df['box_office'].sum()

C:\Users\Bob Thomson\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Bob Thomson\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


489564729284.0

In [371]:
# 14. Clean the budget column in the wiki_movies_df DataFrame.
budget = movies_Step13_df['Budget'].dropna() # Creates list of non-null contents from DataFrame
budget = budget.apply(lambda x: ' '.join(x) if type(x) == list else x)
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
budget[~matches_form_one & ~matches_form_two]
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]
movies_Step13_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
movies_Step13_df.drop('Budget', axis=1, inplace=True)
movies_Step14_df=movies_Step13_df
movies_Step14_df['budget'].sum() # Check on total


C:\Users\Bob Thomson\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


177607020531.0

In [372]:
# 15. Clean the release date column in the wiki_movies_df DataFrame.
release_date = movies_Step14_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

movies_Step14_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
movies_Step14_df.drop('Release date', axis=1, inplace=True)
movies_Step15_df=movies_Step14_df
movies_Step15_df.columns


C:\Users\Bob Thomson\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Index(['url', 'year', 'imdb_link', 'title', 'Based on', 'Starring',
       'Cinematography', 'Running time', 'Country', 'Language', 'Director',
       'Distributor', 'Editor(s)', 'Composer(s)', 'Producer(s)',
       'Production company(s)', 'Writer(s)', 'imdb_id', 'box_office', 'budget',
       'release_date'],
      dtype='object')

In [373]:
# 16. Clean the running time column in the wiki_movies_df DataFrame.
running_time = movies_Step15_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

movies_Step15_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
movies_Step15_df.drop('Running time', axis=1, inplace=True)
movies_Step16_df=movies_Step15_df


C:\Users\Bob Thomson\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [374]:
# Return three variables. The first is the wiki_movies_df DataFrame

# 17. Create the path to your file directory and variables for the three files.
file_dir = "C:/Users/Bob Thomson/ETL Module/"
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [376]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

NameError: name 'wiki_movies_df' is not defined

In [ ]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
movies_Step16_df = wiki_file

In [ ]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

In [ ]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()